# **Setup**
## Recompile Cython Extensions

In [1]:
%cd /scratch/edk202/ngram-prep

%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8

%pip install -e . --no-build-isolation -q

/scratch/edk202/ngram-prep
env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
Note: you may need to restart the kernel to use updated packages.


## Imports

In [1]:
# Auto-reload edited scripts
%load_ext autoreload
%autoreload 2

# NLTK resources
from nltk.corpus import stopwords; stopwords = set(stopwords.words("english"))
from nltk.stem import WordNetLemmatizer; lemmatizer = WordNetLemmatizer()

# Ngram acquisition functions
from ngram_acquire.pipeline.orchestrate import download_and_ingest_to_rocksdb
from ngram_acquire.pipeline.logger import setup_logger

# Ngram processing functions
from pathlib import Path
from ngram_filter.config import PipelineConfig, FilterConfig
from ngram_filter.pipeline.orchestrator import build_processed_db
from common_db.api import open_db

## Set up logging to file

In [2]:
setup_logger(
    db_path="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db",
    console=False,
    rotate=True,
    max_bytes=100_000_000,
    backup_count=5,
    force=True
)

PosixPath('/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db/ngram_download_20250930_001929.log')

# **Download Unigrams and Ingest to RocksDB**

In [ ]:
download_and_ingest_to_rocksdb(
    ngram_size = 1,
    repo_release_id = "20200217",
    repo_corpus_id = "eng",
    db_path_stub = "/vast/edk202/NLP_corpora/Google_Books/",
    file_range = (0, 23),
    random_seed = 21,
    workers = 30,
    use_threads = False,
    ngram_type = "tagged",
    overwrite = True,
    write_batch_size = 100_000,
    open_type = "write:packed24",
    post_compact = True
)

# **Run Processing Pipeline**

In [3]:
src_db = Path("/vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams.db")
dst_db = src_db.parent / "1grams_processed.db"
tmp_dir = src_db.parent / "processing_tmp"

pipeline_config = PipelineConfig(
    src_db=src_db,
    dst_db=dst_db,
    tmp_dir=tmp_dir,
    readers=32,
    ingestors=16,
    work_units_per_reader=1,
    partitioning_sample_rate=0.01,
    prefix_length=4,
    mode="reprocess",
    force_cache_use=False,
    enable_ingest=True,
    delete_after_ingest=True,
    enable_compact=True,
    progress_every_s=60.0,
    output_whitelist_path=dst_db / "whitelist.txt",
    output_whitelist_top_n=40_000
)

filter_config = FilterConfig(
    stop_set=stopwords,
    lemma_gen=lemmatizer,
)

build_processed_db(pipeline_config, filter_config)

N-GRAM FILTER PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Configuration:
════════════════════════════════════════════════════════════════════════════════════════════════════
Pipeline
Run mode: reprocess
Ingest after filtering: True
Compact after ingesting: True
  
Workers
Num Workers: 32
Work units: 32
Profiles: read=read:packed24, write=write:packed24
Buffer: 100,000 items, 128MB
  
Files
Source: /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams.db
Destination: /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_processed.db
Input whitelist: None
Output whitelist: ...ks/20200217/eng/1gram_files/1grams_processed.db/whitelist.txt (top 40,000 keys)

Phase 1: Creating work units...
════════════════════════════════════════════════════════════════════════════════════════════════════
Reprocess - loading cached work units and resetting status
Loaded 32 work units from cache

Phase 2: Proce

As the pipeline runs, the work-unit status is tracked in a SQLite database. Run the following command to check progress.

Execute from the `processing_tmp` directory:
```
python3 -c "
import sqlite3
conn = sqlite3.connect('work_tracker.db')
cur = conn.cursor()
results = cur.execute('SELECT status, COUNT(*) FROM work_units GROUP BY status').fetchall()
status_dict = dict(results)
print(f\"Completed: {status_dict.get('completed', 0)}, Processing: {status_dict.get('processing', 0)}, Pending: {status_dict.get('pending', 0)}\")
"
```

## Run a Manual Compaction

Compaction improves DB read performance. Strong recommendation to run this if compaction was disabled during ingestion.

In [ ]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db"

with open_db(db_path, mode="rw") as db:
    db.compact_all()

In [ ]:
import requests
url = "https://books.storage.googleapis.com/?prefix=ngrams/books/20200217/eng/1-"
resp = requests.get(url)
print(resp.text[:2000])  # First 2000 chars